In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import seaborn as sns
from scipy.stats import pearsonr
import statistics
import copy

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.metrics import f1_score

from sklearn import svm

In [2]:
resistant = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Colon_M\\Colon_mr.txt', header=None, delimiter=r"\s+")

control = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Colon_M\\Colon_mc.txt', header=None, delimiter=r"\s+")

known = pd.read_csv("C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Colon_M\\Known_Colon_M.csv")

name = pd.read_csv('C:\\Users\\Muskan Agarwal\\Documents\\CB\\Dissertation\\Dataset\\akashM.techdatasets\\Colon_M\\m colonmiRNA names.txt', header=None)


In [3]:
resistant = resistant.T
resistant

,0,1,2,3,4,5,6,7,8,9,...,713,714,715,716,717,718,719,720,721,722
0,11037.5,-7.46623,5363.19,3.32888,969.683,0.565693,3221.32,9.02072,5287.27,2.47449,...,-12.4572,-9.982900,2.58784,19.7765,-12.8541,315.573,-7.00533,6.27522,575.526,-5.119980
1,23310.7,2.46680,11332.90,5.41868,1911.700,-12.401500,6884.39,14.40860,11895.70,10.64720,...,13.1608,18.536800,-4.09915,92.8066,-24.5146,686.684,7.79242,-5.74272,855.181,-27.198500
2,18546.7,-7.18517,7330.94,7.55638,1857.150,-9.353450,5465.06,10.34910,7317.24,16.15740,...,1.3897,0.255608,-9.90477,98.9957,-13.0292,763.165,-2.60348,1.06545,912.944,0.870409


In [4]:
control = control.T


In [5]:
known_control, known_resistant, index_name, index_known = [], [], [], []

for i in range(len(known)):
    for j in range(len(name)):
        if known[known.columns[1]][i] == name[0][j]:
            known_control.append(control[j])
            known_resistant.append(resistant[j])
            index_name.append(j)
            index_known.append(i)
            break


In [6]:
len(known_resistant)

9

In [8]:
columns = len(control.T)
mr = []
for i in range(columns):
    mr.append(resistant[i].mean())

mc = []
for i in range(columns):
    mc.append(control[i].mean())


In [9]:
known_mc, known_mr = [], []

for i in range(len(known)):
    for j in range(len(name)):
        if known[known.columns[1]][i] == name[0][j]:
            known_mc.append(mc[j])
            known_mr.append(mr[j])


In [10]:
D, F, corr, corr_known, D_known, F_known, scaled_corr, scaled_corr_known  = [], [], [], [], [], [], [], []
for x in range(columns):
    D.append(abs(mr[x] - mc[x]))
    
    if mc[x] >= mr[x]:
        F.append((mc[x] - mr[x]) / mc[x])
    else:
        F.append((mr[x] - mc[x]) / mr[x])
        
    corr.append(control[x].corr(resistant[x]))
    scaled_corr.append((1-corr[x])/2)
    
    
    
for x in range(len(known_control)):
    D_known.append(abs(known_mr[x] - known_mc[x]))
    
    if known_mc[x] >= known_mr[x]:
        F_known.append((known_mc[x] - known_mr[x]) / known_mc[x])
    else:
        F_known.append((known_mr[x] -known_mc[x]) / known_mr[x])
    
    corr_known.append(known_control[x].corr(known_resistant[x]))
    scaled_corr_known.append((1-corr_known[x])/2 )

In [11]:
P, A, B, C, M = [], [], [], [], []
for p in np.arange(0.1, 1.6, 0.1):
    for a in np.arange(.1, 1, 0.01):
        for b in np.arange(0.1, 1, 0.01):
            for c in np.arange(0.1, 1, 0.01):
                l = []
                for x in range(len(known_control)):
                     l.append((a*D_known[x] + b*(abs(F_known[x])**p) + c*scaled_corr_known[x])/(a + b + c))
                P.append(p)
                A.append(a)
                B.append(b)
                C.append(c)
                M.append(sum(l) / len(l))
                
min_avg_rank = M.index(max(M))
p, a, b, c = P[min_avg_rank], A[min_avg_rank], B[min_avg_rank], C[min_avg_rank]
print(f"min_avg_rank = {min_avg_rank}, p = {p}, a= {a}, b={b}, c={c} ")


min_avg_rank = 720900, p = 0.1, a= 0.9899999999999995, b=0.1, c=0.1 


In [13]:
Q = []
for x in range(len(control.T)):
    Q.append((a*D[x] + b*(abs(F[x])**p) + c*scaled_corr[x])/(a + b + c))

In [15]:
G = [(x, i) for i, x in enumerate(Q)]
G.sort(reverse = True)
E = G[:30]

In [18]:

for i in range(len(E)):
    for j in range(len(index_name)):
        if E[i][1] == index_name[j]:
            print(f'name:{name[0][index_name[j]]}, index in name:{index_name[j]}, rank: {i}')
            break
            

name:hsa-miR-193b, index in name:163, rank: 11
name:hsa-miR-210, index in name:228, rank: 12
name:hsa-miR-365, index in name:331, rank: 15
name:hsa-miR-455-3p, index in name:395, rank: 17
name:hsa-miR-27b, index in name:245, rank: 18


In [19]:
l = int(len(F) * 0.05)
H = G[ : l]

In [20]:
mat = np.zeros((len(control) + len(resistant), len(H) + 1), dtype = 'float64')
for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
        if i < len(resistant):
            if j != mat.shape[1] - 1:
                mat[i][j] = resistant[H[j][1]][i]
            else:
                mat[i][j] = 0
        else:
            if j != mat.shape[1] - 1:
                mat[i][j] = control[H[j][1]][i - len(resistant)]
            else:
                mat[i][j] = 1

In [22]:
X, y = mat[:,:-1], mat[:,-1:]

In [23]:
X.shape, y.shape

((6, 36), (6, 1))

In [34]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut

# Create the Random Forest classifier
clf = RandomForestClassifier(n_estimators=2, random_state=27)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)


Accuracy: 0.8333333333333334
Specificity: 1.0
Sensitivity: 0.6666666666666666
MCC: 0.7071067811865476
F1-score: 0.8


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change th

In [37]:
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, f1_score
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Create the SVM classifier
clf = svm.SVC(kernel='linear', C=1, random_state=42)

# Perform Leave-One-Out Cross-Validation
predictions = []
true_labels = []
for train_index, test_index in LeaveOneOut().split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the classifier on the training data
    clf.fit(X_train, y_train)
    
    # Make predictions on the test data
    y_pred = clf.predict(X_test)
    
    # Collect the predictions and true labels
    predictions.append(y_pred[0])
    true_labels.append(y_test[0])

# Calculate accuracy, specificity, sensitivity, MCC, and F1-score
accuracy = accuracy_score(true_labels, predictions)
tn, fp, fn, tp = confusion_matrix(true_labels, predictions).ravel()
specificity = tn / (tn + fp)
sensitivity = tp / (tp + fn)
mcc = matthews_corrcoef(true_labels, predictions)
f1 = f1_score(true_labels, predictions)

# Print the results
print("Accuracy:", accuracy)
print("Specificity:", specificity)
print("Sensitivity:", sensitivity)
print("MCC:", mcc)
print("F1-score:", f1)
# print("Avg accuracy:", np.mean(scores))


Accuracy: 0.8333333333333334
Specificity: 1.0
Sensitivity: 0.6666666666666666
MCC: 0.7071067811865476
F1-score: 0.8


c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\users\muskan agarwal\appdata\local\programs\python\python37\lib\site-packages\sklearn\ut